# Natural Language Processing with Disaster Tweets









In this competition, you’re challenged to build a machine learning model that predicts which Tweets are about real disasters and which one’s aren’t. You’ll have access to a dataset of 10,000 tweets that were hand classified.

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
ROOT = 'drive/MyDrive/DisasterTweets_bert/'
TRAIN_DATA_PATH = ROOT + 'data/train.csv'
TEST_DATA_PATH = ROOT + 'data/test.csv'
LOG_DIR = ROOT + 'tensorboard/tb_bert'
MODEL_PATH = ROOT + 'model/bert_checkpoint.h5'
BERT_CHECKPOINT_PATH = ROOT + 'model/checkpoint'

In [4]:
!pip install transformers
!mkdir data


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 80.5 MB/s eta 0:00:00


In [5]:
import os
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
import keras

## BERT

In [6]:
num_classes = 2
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)
#bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
train_df = pd.read_csv(TRAIN_DATA_PATH)
train_df = train_df.drop(['id', 'keyword', 'location'], axis=1)
train_df = train_df.rename(columns={"text": "input_ids", "target": "labels"})
EXAMPLE_SIZE = train_df.shape[0]

FileNotFoundError: ignored

In [ ]:
in_batch_encoding = bert_tokenizer(train_df.input_ids.tolist(), padding=True, return_tensors="tf")
labels = tf.convert_to_tensor(train_df['labels'])
labels = tf.expand_dims(labels, axis=-1)
#print(tf.unique(labels)[0])
print(labels)
len(train_df.labels.unique())
#targ_batch_encoding = bert_tokenizer(train_df.labels.astype(str).tolist(),  max_length=2, return_tensors="tf")

In [ ]:
ds_train_bert = tf.data.Dataset.from_tensor_slices(({'input_ids' : in_batch_encoding.input_ids,
                                                     'attention_mask' : in_batch_encoding.attention_mask},
                                                    labels))

In [ ]:
# split train/val set
def split_dataset(ds, ds_size, val_split=0.1):
    #ds_size = tf.data.experimental.cardinality(train_ds).numpy()
    train_size = int((1 - val_split) * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    
    return train_ds, val_ds
    
# prefetch, shuffle and batch the dataset
def dataset_prepare(dataset, batch_size=32, training=True):
    dataset = dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    if training:
        dataset = dataset.shuffle(1*1024)
    dataset = dataset.batch(batch_size)
    return dataset

dataset = dataset_prepare(ds_train_bert)
ds_train, ds_val = split_dataset(dataset, EXAMPLE_SIZE)

In [ ]:
aaa = iter(ds_train)
bbb= next(aaa)

In [ ]:
print(bert_tokenizer.decode(bbb[0]['input_ids'][4]))
# print(bert_tokenizer.decode(bbb[0]['input_ids'][3], skip_special_tokens=True))
# print(bbb[0]['attention_mask'][3])
# print((bbb[1][3]))
#bbb[1]

In [ ]:
# def build_BERT_classifier(bert_model):
#   encoder_inputs = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
#   outputs = bert_model(encoder_inputs)
#   net = outputs['pooled_output']
#   net = tf.keras.layers.Dropout(0.1)(net)
#   net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
#   return tf.keras.Model(text_input, net)
# bert_model = build_BERT_classifier(bert_model)

In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint(filepath=MODEL_PATH,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),
             keras.callbacks.TensorBoard(log_dir=LOG_DIR),
             keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]

print('\nBert Model',bert_model.summary())

bert_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# recommended learning rate for Adam 5e-5, 3e-5, 2e-5
bert_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)

bert_model.compile(loss=bert_loss,optimizer=bert_optimizer,metrics=metric)

In [ ]:
history=bert_model.fit(
    ds_train,
    batch_size=32,
    epochs=30,
    validation_data=ds_val,
    callbacks=callbacks)


In [ ]:

bert_checkpoint = tf.train.Checkpoint(bert_model)
bert_checkpoint_save = bert_checkpoint.save(BERT_CHECKPOINT_PATH)

In [ ]:
# bert_checkpoint = tf.train.Checkpoint(bert_model)
# bert_checkpoint.restore(bert_checkpoint_save)

In [ ]:
tf.saved_model.save(bert_model, MODEL_PATH)

In [ ]:
predict_encoding = bert_tokenizer("i like swimming", return_tensors="tf")
predict_mask = predict_encoding.attention_mask
predict_inputs = predict_encoding.input_ids 
predict_token_type_ids = predict_encoding.token_type_ids
prediction = bert_model.predict([predict_inputs, predict_mask, predict_token_type_ids])
#prediction_labels = prediction.argmax(axis=1)
print(prediction.logits.argmax())

# Evaluation

In [ ]:
bert_model.evaluate(dataset)

# Testing

In [ ]:
test_df = pd.read_csv(TEST_DATA_PATH)
EXAMPLE_SIZE = train_pd.shape[0]
test_df.sort_values(by=['id'])
test_df = test_df.drop(['keyword', 'location'], axis=1)
test_df = test_df.rename(columns={"text": "input_ids"})

In [ ]:
pd.Series(test_df.labels).hist(bins = 4)

In [ ]:
tweet_len = [len(i.split()) for i in test_df.text]
pd.Series(tweet_len).hist(bins = 30)

In [ ]:
test_df.head()

In [ ]:
in_batch_encoding = bert_tokenizer(test_df.input_ids.tolist(), padding=True, return_tensors="tf")
ds_test_bert = tf.data.Dataset.from_tensor_slices({'input_ids' : in_batch_encoding.input_ids,
                                                     'attention_mask' : in_batch_encoding.attention_mask})
dataset = dataset_prepare(ds_test_bert, training=False)

In [ ]:
result = bert_model.predict(dataset)

In [ ]:
result.logits[:5]

In [ ]:
bert_prediction = tf.sigmoid(result.logits)

In [ ]:
bert_prediction[:5]

In [ ]:
bert_result = tf.argmax(bert_prediction, axis=-1)

In [ ]:
bert_result[:5]

In [ ]:
df_lables = pd.DataFrame(bert_result)
df_lables.columns = ['target']
test_df = test_df.drop(['input_ids'], axis=1)
df_submission = pd.concat([df_lables, test_df], axis=1)
df_submission.to_csv('data/submission.csv', index=False)

In [ ]:
df_submission

In [ ]:
result

In [ ]:
%load_ext tensorboard

In [ ]:
log_dir='tensorboard/tb_bert'
%tensorboard --logdir {log_dir} --host localhost --port 8083

# Appendix

## Other user's code

https://swatimeena989.medium.com/bert-text-classification-using-keras-903671e0207d

https://atheros.ai/blog/text-classification-with-transformers-in-tensorflow-2

https://www.tensorflow.org/text/tutorials/classify_text_with_bert

In [ ]:
import pandas as pd
# Recommended tensorflow version is <= 2.1.0, otherwise F1 score function breaks
import tensorflow as tf
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
from transformers import TFRobertaForSequenceClassification
from transformers import RobertaTokenizer

# Load your Dataset
train_tweets = pd.read_csv('train_tweets.csv')
test_tweets = pd.read_csv('test_tweets.csv')

test_tweets['label'] = 0

training_sentences, testing_sentences = train_test_split(train_tweets[['tweet', 'label']],
                                                         test_size=0.2)

roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# can be up to 512 for BERT
max_length = 512

# the recommended batches size for BERT are 32,64 ... however on this dataset we are overfitting quite fast
# and smaller batches work like a regularization.
# You might play with adding another dropout layer instead.

batch_size = 64

def convert_example_to_feature(review):
    # combine step for tokenization, WordPiece vector mapping and will
    # add also special tokens and truncate reviews longer than our max length
    return roberta_tokenizer.encode_plus(review,
                                 add_special_tokens=True,  # add [CLS], [SEP]
                                 max_length=max_length,  # max length of the text that can go to RoBERTa
                                 pad_to_max_length=True,  # add [PAD] tokens at the end of sentence
                                 return_attention_mask=True,  # add attention mask to not focus on pad tokens
                                 )

# map to the expected input to TFRobertaForSequenceClassification, see here
def map_example_to_dict(input_ids, attention_masks, label):
    return {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
           }, label

def encode_examples(ds, limit=-1):
    # Prepare Input list
    input_ids_list = []
    attention_mask_list = []
    label_list = []

    if (limit > 0):
        ds = ds.take(limit)

    for review, label in tfds.as_numpy(ds):
        bert_input = convert_example_to_feature(review.decode())
        input_ids_list.append(bert_input['input_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])

    return tf.data.Dataset.from_tensor_slices((input_ids_list,
                                               attention_mask_list,
                                               label_list)).map(map_example_to_dict)

training_sentences_modified = tf.data.Dataset.from_tensor_slices((training_sentences['tweet'],
                                                                  training_sentences['label']))

testing_sentences_modified = tf.data.Dataset.from_tensor_slices((testing_sentences['tweet'],
                                                                 testing_sentences['label']))

ds_train_encoded = encode_examples(training_sentences_modified).shuffle(10000).batch(batch_size)
ds_test_encoded = encode_examples(testing_sentences_modified).batch(batch_size)



learning_rate = 6e-5
number_of_epochs = 6

class ModelMetrics(tf.keras.callbacks.Callback):

    def on_epoch_end(self, batch, logs={}):
        y_val_pred = tf.nn.softmax(self.model.predict(ds_test_encoded))
        y_pred_argmax = tf.math.argmax(y_val_pred, axis=1)
        testing_sentences['predicted'] = y_pred_argmax
        f1_s = f1_score(testing_sentences['label'], testing_sentences['predicted'])
        print('\n f1 score is :', f1_s)

metrics = ModelMetrics()

# model initialization
model = TFRobertaForSequenceClassification.from_pretrained("roberta-base")
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)

# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
model.fit(ds_train_encoded, epochs=number_of_epochs, 
          validation_data=ds_test_encoded, callbacks=[metrics])


# submittion file
# submission_sentences_modified = tf.data.Dataset.from_tensor_slices((test_tweets['tweet'],
#                                                                  test_tweets['label']))
# ds_submission_encoded = encode_examples(submission_sentences_modified).batch(batch_size)
# submission_pre = tf.nn.softmax(model.predict(ds_submission_encoded))
# submission_pre_argmax = tf.math.argmax(submission_pre, axis=1)
# test_tweets['label'] = submission_pre_argmax
# test_tweets[['id', 'label']].to_csv('submission.csv', index=False, header=True)